In [1]:
'''
    Code written by Yacine Mahdid 2019-11-24 for teaching purposes
    Ressources: 
        https://sefiks.com/2017/11/20/a-step-by-step-id3-decision-tree-example/
        https://en.wikipedia.org/wiki/ID3_algorithm
        https://www.displayr.com/how-is-splitting-decided-for-decision-trees/
'''
# Data science import
import numpy as np

# Visualization import
import matplotlib.pyplot as plt

# General ML Import
from sklearn import svm
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

# Specific Decision Tree import
from id3 import Id3Estimator

id3_estimator = Id3Estimator()
svm_estimator = svm.SVC(kernel='linear', probability=True)
dataset = load_breast_cancer()

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=.5,random_state=0)

y_score_id3 = id3_estimator.fit(X_train, y_train).predict(X_test)
y_score_svm = svm_estimator.fit(X_train, y_train).decision_function(X_test)

fpr_id3, tpr_id3, _  = roc_curve(y_test, y_score_id3)
roc_auc_id3 = auc(fpr_id3, tpr_id3)

fpr_svm, tpr_svm, _  = roc_curve(y_test, y_score_svm)
roc_auc_svm = auc(fpr_svm, tpr_svm)

plt.figure()
plt.plot(fpr_id3, tpr_id3, color='darkorange',lw=2, label='ROC curve Id3 (area = %0.2f)' % roc_auc_id3)
plt.plot(fpr_svm, tpr_svm, color='red',lw=2, label='ROC curve SVM (area = %0.2f)' % roc_auc_svm)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating ID3 vs SVM')
plt.legend(loc="lower right")
plt.show()



C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


<Figure size 640x480 with 1 Axes>

In [12]:
dataset.target


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [ ]:
'''
    Code written by Yacine Mahdid 2019-11-24 for teaching purposes
    Ressources: 
        https://sefiks.com/2017/11/20/a-step-by-step-id3-decision-tree-example/
        https://en.wikipedia.org/wiki/ID3_algorithm
'''

import numpy as np

# Here we assume that the values are all numerical

# Structure Setup
class Tree:    
    # Constructor for leaves
    def __init__(self,data):        
        # This is set only for leaves node
        self.data = data
        self.isLeave = False
        
    # Constructor for splitting node
    def __init__(self,splitDimension, splitValue,left,right):
        self.splitDimension = splitDimension
        self.splitValue = splitValue
        self.left = left
        self.right = right

# Entropy(S) = Sum for each classes in S of (-p(s)*log(p(s)))
def calculate_entropy(X,y,c_i,value):
    
    # We are separating the dataset X in left and right
    left = []
    right = []
    
    (num_x) = np.shape(X)
    for x in X:
        if(x <= value):
            left.append(x)
        else:
            right.append(x)
    
    # will calculate the entropy on the left(doesn't matter)
    for y_test in [0,1]:
        p = len(left)/len(X)
        e = e + -p*np.log(p)
        

# will tell us if the given tree is a leaf or a node
def is_leaf(y):
    if(y.count(y[0]) == len(y)):
        return True
    return False

# Finding the best split with all the data point (we will use the 10th percentile 0 - 0.1 - 0.2 - etc. 1.0)
# for debugging let's use all the values and a
# here the dataset is not a leaf
def find_optimal_split(X, y):
    # This need to return both the splitDimension splitValue 
    # This will hold the best splitDimension and splitValue
    splitDimension = -1
    splitValue = -1
    entropy = -1
    
    (num_value, num_class) = np.shape(X)
    for c_i in range(0, num_class):
        for v_i in range(0, num_value):
            value = X[c_i,v_i]
            current_entropy = calculate_entropy(X[c_i,:],y,value)
            
            # keep the best entropy
            if(entropy == -1 or current_entropy < entropy):
                splitDimension = c_i
                splitValue = value
    
    # splitDimension is an index
    # splitValue is the value for that class 
    return (splitDimension, splitValue)
    
# We calculate the entropy for each of the splitDimension and splitValues
# the lowest entropy wins

